<a href="https://colab.research.google.com/github/Ayikanying-ux/Getting_started_-with_deep_learning/blob/main/automatic_differentiation_and_gradients.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automatic Differentiation and Gradients
Automatic differentiation is useful for implementing machine learning algorithms such as backpropagation for training neural networks

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

### Computing gradients
To differentiate automatically, TensorFlow needs to remember what operations happen in what order during the forward pass, Then, during the backward pass, TensorFlow traverses the list of operations in reserve order to compute gradients

### Gradient tapes
TensorFlow provides the tf.GradientTape API for automatic differentiation; that is, computing the gradient of a computation with respect to some inputs, usually tf.Variables. TensorFlow "records" relevant operations executed inside the context of a tf.GradientTape onto a "tape". TensorFlow then used that tape to compute the gradients of a "recorded" computation using reverse mode differentiation.

In [2]:
x = tf.Variable(3.0)

with tf.GradientTape() as tape:
  y = x**2



---
Once you've recorded some operations, use GradientTape.gradient(target, sources) to calculate teh gradient of some target (often a loss) relative to some source (often the model's varibles)

In [4]:
dy_dx = tape.gradient(y, x)
dy_dx.numpy()

6.0